In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ti = pd.read_csv('C:/rcode/train.csv')

In [2]:
ti.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [3]:
ti = ti.drop(['Name','SibSp','Parch','Ticket','Fare','Cabin'], axis=1)

In [4]:
ti

,PassengerId,Survived,Pclass,Sex,Age,Embarked
0,1,0,3,male,22.0,S
1,2,1,1,female,38.0,C
2,3,1,3,female,26.0,S
3,4,1,1,female,35.0,S
4,5,0,3,male,35.0,S
...,...,...,...,...,...,...
886,887,0,2,male,27.0,S
887,888,1,1,female,19.0,S
888,889,0,3,female,NaN,S
889,890,1,1,male,26.0,C


In [5]:
ti.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
Embarked         2
dtype: int64

In [6]:
ti['Age'] = ti['Age'].fillna(ti['Age'].mean())
ti['Embarked'] = ti['Embarked'].fillna('S')

In [7]:
from sklearn.preprocessing import LabelEncoder
item = ti['Embarked']
encoder = LabelEncoder()
encoder.fit(item)
labels = encoder.transform(item)
ti['Embarked'] = labels

In [8]:
sex = ti['Sex']
encoder = LabelEncoder()
encoder.fit(sex)
sexs = encoder.transform(sex)
ti['Sex'] = sexs

In [9]:
ti

,PassengerId,Survived,Pclass,Sex,Age,Embarked
0,1,0,3,1,22.000000,2
1,2,1,1,0,38.000000,0
2,3,1,3,0,26.000000,2
3,4,1,1,0,35.000000,2
4,5,0,3,1,35.000000,2
...,...,...,...,...,...,...
886,887,0,2,1,27.000000,2
887,888,1,1,0,19.000000,2
888,889,0,3,0,29.699118,2
889,890,1,1,1,26.000000,0


In [10]:
sv = pd.DataFrame()

In [11]:
sv = ti['Survived']
ti= ti.drop('Survived',axis=1)

In [12]:
sv

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [13]:
ti

,PassengerId,Pclass,Sex,Age,Embarked
0,1,3,1,22.000000,2
1,2,1,0,38.000000,0
2,3,3,0,26.000000,2
3,4,1,0,35.000000,2
4,5,3,1,35.000000,2
...,...,...,...,...,...
886,887,2,1,27.000000,2
887,888,1,0,19.000000,2
888,889,3,0,29.699118,2
889,890,1,1,26.000000,0


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(ti, sv, \
                                                  test_size=0.2, random_state=11)

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression()

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train , y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train , y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train , y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

DecisionTreeClassifier 정확도: 0.7542
RandomForestClassifier 정확도:0.8101
LogisticRegression 정확도: 0.8492


In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, ti , sv , cv=5)
for iter_count,accuracy in enumerate(scores):
    print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))

print("평균 정확도: {0:.4f}".format(np.mean(scores)))

교차 검증 0 정확도: 0.6369
교차 검증 1 정확도: 0.7753
교차 검증 2 정확도: 0.7697
교차 검증 3 정확도: 0.7584
교차 검증 4 정확도: 0.7697
평균 정확도: 0.7420


In [17]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,3,5,10],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

grid_dclf = GridSearchCV(dt_clf , param_grid=parameters , scoring='accuracy' , cv=5)
grid_dclf.fit(X_train , y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test , dpredictions)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8104
테스트 세트에서의 DecisionTreeClassifier 정확도 : 0.8659


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(20,5))